In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from datetime import datetime
import seaborn as sns
%matplotlib inline

In [9]:
import os
os.chdir("C:\\Users\\vsm397\\Desktop\\Vir's Personal documents\\Python Class- Summer 2018\\Python Project")

In [10]:
weather = pd.read_csv('austin_weather.csv').drop(['DewPointHighF','DewPointLowF','HumidityHighPercent','HumidityLowPercent',
                                               'SeaLevelPressureHighInches','SeaLevelPressureLowInches','VisibilityHighMiles',
                                               'VisibilityLowMiles','WindHighMPH','WindGustMPH','HumidityAvgPercent'],1)

trips=pd.read_csv('austin_bikeshare_trips.csv')
station=pd.read_csv('austin_bikeshare_stations_w_zipcode.csv')

In [11]:
weather['Rain']=np.where(weather.Events.str.contains('Rain'), 1, 0)
weather['Fog']=np.where(weather.Events.str.contains('Fog'), 1, 0)
weather['Thunderstorm']=np.where(weather.Events.str.contains('Thunderstorm'), 1, 0)
weather['Snow']=np.where(weather.Events.str.contains('Snow'), 1, 0)
weather['None']=np.where(weather.Events.isnull, 1, 0)
weather.Date=pd.to_datetime(weather.Date)
weather=weather.set_index('Date').drop('Events',1)

In [13]:
ind = pd.DatetimeIndex(trips.start_time)
trips['Date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour
by_date = trips.pivot_table('trip_id', aggfunc='count',index='Date',columns='subscriber_type', )
by_date = trips.groupby(['Date', 'subscriber_type'])['trip_id'].count()
by_date.name = 'count'
by_date = by_date.reset_index('subscriber_type')
trips_2=trips[trips.year.isin([2014,2015,2016])]
trip_weather=trips_2.merge(weather,left_on='Date',right_on='Date',
                         suffixes=['_trips','_weather']).drop(['SeaLevelPressureAvgInches'],1)

In [14]:
trip_weather["start_time"] = trip_weather["start_time"].map(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_weather["hours"] = trip_weather["start_time"].map(lambda x : x.hour)


In [17]:
val=''
def time_of_day(x):
    global val
    if (x>=6)&(x<10):
        val='morning rush hours'
    elif (x>=15)&(x<19):
        val='afternoon ruch hours'
    elif (x>=10)&(x<15):
        val='noon'
    elif (x>=19)|(x<6):
        val='night'
    return val
trip_weather['Time of the day'] = trip_weather['hours'].apply(lambda a: time_of_day(a))



In [18]:
trip_zipcode=trip_weather.merge(station,left_on='start_station_id',right_on='station_id')

In [19]:
trip_zipcode.loc[trip_zipcode['zipcode'] != 78705, 'University_vicinity'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'University_vicinity'] = 1
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,Time of the day,Unnamed: 0,latitude,location,longitude,name,station_id,status,zipcode,University_vicinity
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0


In [20]:
holidays = ['2014-01-01', '2015-01-01', '2016-01-01','2014-01-18', '2014-01-19','2014-01-20','2015-01-17', 
            '2015-01-18','2015-01-19','2016-01-16', '2016-01-17','2016-01-18','2014-02-15', '2014-02-16', '2014-02-17',
            '2015-02-14', '2015-02-15', '2015-02-16','2016-02-13', '2016-02-14', '2016-02-15',
            '2014-05-24', '2014-05-25', '2014-05-26','2015-05-23', '2015-05-24', '2015-05-25','2016-05-28', '2016-05-29', 
            '2016-05-30','2014-07-04', '2014-07-05', '2014-07-06','2015-07-03', '2015-07-04', '2015-07-05',
            '2016-07-02', '2016-07-03', '2016-07-04','2014-08-30', '2014-08-31', '2014-09-01','2015-09-05', '2015-09-06',
            '2015-09-07','2016-09-03', '2016-09-04', '2016-09-05','2014-10-11', '2014-10-12', '2014-10-13',
            '2015-10-10', '2015-10-11', '2015-10-12','2016-10-08', '2016-10-09', '2016-10-10','2014-11-11', '2015-11-11',
            '2016-11-11','2014-11-27', '2014-11-28', '2014-11-29', '2014-11-30','2015-11-26', '2015-11-27', '2015-11-28',
            '2015-11-29','2016-11-24', '2016-11-25', '2016-11-26', '2016-11-27','2014-12-25', '2014-12-26', '2014-12-27',
            '2014-12-28','2015-12-25', '2015-12-26', '2015-12-27', '2015-12-28','2016-12-24', '2016-12-25', '2016-12-26',
            '2016-12-27','2014-03-07', '2014-03-16','2015-03-13', '2015-03-22', '2016-12-27','2014-03-07', '2014-03-16',
            '2015-03-13', '2015-03-22','2016-03-11', '2016-03-20','2014-10-03', '2014-10-05','2014-10-10', '2014-10-12',
            '2015-10-02', '2015-10-04','2015-10-09', '2015-10-11','2016-09-30', '2016-10-02','2016-10-07', '2016-10-09']
trip_zipcode.loc[-trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 0
trip_zipcode.loc[trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 1
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,Unnamed: 0,latitude,location,longitude,name,station_id,status,zipcode,University_vicinity,Holiday_flag
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0


In [26]:
trip_zipcode['weekday'] = trip_zipcode['Date'].apply(lambda x: x.weekday())
trip_zipcode.loc[trip_zipcode['weekday'].isin([1,2,3,4,5]), 'Weekday_flag'] = 1
trip_zipcode.loc[trip_zipcode['weekday'].isin([6,7]), 'Weekday_flag'] = 0
trip_zipcode

,bikeid,checkout_time,duration_minutes,month,start_station_id,trip_id,Date,TempAvgF,Rain,Fog,...,Unnamed: 0,latitude,longitude,name,status,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag
0,8.0,19:12:00,41,3.0,2536.0,9900082882,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
1,371.0,19:12:00,6,3.0,2536.0,9900082857,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
2,922.0,19:12:00,31,3.0,2536.0,9900082964,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
3,991.0,21:12:00,25,3.0,2536.0,9900083332,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
4,822.0,21:12:00,0,3.0,2536.0,9900083340,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
5,894.0,17:12:00,7,3.0,2536.0,9900082478,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
6,974.0,12:12:00,7,3.0,2536.0,9900081225,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
7,284.0,19:12:00,11,3.0,2536.0,9900083042,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
8,54.0,23:12:00,59,3.0,2536.0,9900083609,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0
9,881.0,18:12:00,6,3.0,2536.0,9900082683,2015-03-19,73,0,0,...,7,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0


In [32]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Tourist_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Tourist_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Tourist_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Tourist_Location'] = 0
trip_zipcode

,bikeid,checkout_time,duration_minutes,month,start_station_id,trip_id,Date,TempAvgF,Rain,Fog,...,latitude,longitude,name,status,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag,Tourist_Location
0,8.0,19:12:00,41,3.0,2536.0,9900082882,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
1,371.0,19:12:00,6,3.0,2536.0,9900082857,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
2,922.0,19:12:00,31,3.0,2536.0,9900082964,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
3,991.0,21:12:00,25,3.0,2536.0,9900083332,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
4,822.0,21:12:00,0,3.0,2536.0,9900083340,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
5,894.0,17:12:00,7,3.0,2536.0,9900082478,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
6,974.0,12:12:00,7,3.0,2536.0,9900081225,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
7,284.0,19:12:00,11,3.0,2536.0,9900083042,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
8,54.0,23:12:00,59,3.0,2536.0,9900083609,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0
9,881.0,18:12:00,6,3.0,2536.0,9900082683,2015-03-19,73,0,0,...,30.26461,-97.73049,Waller & 6th St.,closed,78702,0.0,0.0,3,1.0,1.0


In [35]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Residential_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Residential_Location'] = 0

In [37]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Pop_Age_15-24'] = 2862
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Pop_Age_15-24'] = 2881
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Pop_Age_15-24'] = 1650
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Pop_Age_15-24'] = 6345
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Pop_Age_15-24'] = 24662
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Pop_Age_15-24'] = 7119
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Pop_Age_15-24'] = 2529
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Pop_Age_15-24'] = 3604
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Pop_Age_15-24'] = 2455

In [38]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Walk_to_Work'] = 745
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Walk_to_Work'] = 376
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Walk_to_Work'] = 449
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Walk_to_Work'] = 656
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Walk_to_Work'] = 2801
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Walk_to_Work'] = 628
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Walk_to_Work'] = 218
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Walk_to_Work'] = 158
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Walk_to_Work'] = 222

In [39]:
trip_zipcode

,bikeid,checkout_time,duration_minutes,month,start_station_id,trip_id,Date,TempAvgF,Rain,Fog,...,status,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag,Tourist_Location,Residential_Location,Pop_Age_15-24,Walk_to_Work
0,8.0,19:12:00,41,3.0,2536.0,9900082882,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
1,371.0,19:12:00,6,3.0,2536.0,9900082857,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
2,922.0,19:12:00,31,3.0,2536.0,9900082964,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
3,991.0,21:12:00,25,3.0,2536.0,9900083332,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
4,822.0,21:12:00,0,3.0,2536.0,9900083340,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
5,894.0,17:12:00,7,3.0,2536.0,9900082478,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
6,974.0,12:12:00,7,3.0,2536.0,9900081225,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
7,284.0,19:12:00,11,3.0,2536.0,9900083042,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
8,54.0,23:12:00,59,3.0,2536.0,9900083609,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
9,881.0,18:12:00,6,3.0,2536.0,9900082683,2015-03-19,73,0,0,...,closed,78702,0.0,0.0,3,1.0,1.0,1.0,1650.0,376.0
